In [238]:
# Import libraries
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import csv
import pandas as pd

In [239]:
# Setup project target, example: Data Science Junior Jobs in Florianopolis
url = 'https://www.linkedin.com/jobs/search?keywords=junior%20data%20analyst&location=Florianopolis&refresh=true'
print("URL that generates the initial CSV list: ", url)

URL that generates the initial CSV list:  https://www.linkedin.com/jobs/search?keywords=junior%20data%20analyst&location=Florianopolis&refresh=true


In [240]:
# Setup chromedriver 
try:
    wd = webdriver.Chrome()
except:
    wd = webdriver.Chrome(ChromeDriverManager().install())
wd.get(url)

# Loop verifying if a redirect has occurred and then undoes it.
while 'authwall' in wd.current_url:
    print(wd.current_url)
    time.sleep(1)
    wd.get(url)



/tmp/ipykernel_29990/1741417621.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(ChromeDriverManager().install())


In [241]:
# Get the number of jobs available
no_of_jobs = int(wd.find_element(By.CSS_SELECTOR, value='h1>span').get_attribute('innerText'))
print("Number of jobs found: ",no_of_jobs-2)

Number of jobs found:  218


In [242]:
# Browse all the jobs
i = 2
while i <= int(no_of_jobs/25)+1: 
    wd.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    i = i + 1
    try:
        wd.find_element(By.XPATH, value='/html/body/div[1]/div/main/section[2]/button').click()
        time.sleep(5)
    except:
        pass
        time.sleep(5)

    

In [243]:
# Find all the jobs
job_lists = wd.find_element(By.CLASS_NAME, value='jobs-search__results-list')
jobs = job_lists.find_elements(By.TAG_NAME, value='li') # return a list

In [244]:
# Initalizing job details into dataframe
job_id = []
job_title = []
company_name = []
location = []
date = []
job_link = []
jd = []
seniority = []
emp_type = []
job_func = []
industries = []

In [245]:
# Loading job details into dataframe
for job in jobs:
        job_id0 = job.find_element(By.CLASS_NAME, value='base-card').get_attribute('data-entity-urn')
        job_id.append(job_id0)
 
        job_title0 = job.find_element(By.CSS_SELECTOR, value='h3').text
        job_title.append(job_title0)
 
        company_name0 = job.find_element(By.CSS_SELECTOR, value='h4').text
        company_name.append(company_name0)
 
        location0 = job.find_element(By.CLASS_NAME, value='job-search-card__location').text
        location.append(location0)
 
        date0 = job.find_element(By.CSS_SELECTOR, value='div>div>time').get_attribute('datetime')
        date.append(date0)
 
        job_link0 = job.find_element(By.CSS_SELECTOR, value='a').get_attribute('href')
        job_link.append(job_link0) 


        # Creating the soon to be filled columns of each job
        
        jd.append(" ")
        seniority.append(" ")
        emp_type.append(" ")
        job_func.append(" ")
        industries.append(" ")



In [246]:
print("Number of extracted job listing links: ",len(jobs))

Number of extracted job listing links:  220


In [247]:
# Load data into DataFrame for further analysis
job_data = pd.DataFrame({'ID': job_id,
                         'Date': date,
                         'Company': company_name,
                         'Title': job_title,
                         'Location': location,
                         'Link': job_link,
                         'Description': jd,
                         'Level': seniority,
                         'Type': emp_type,
                         'Function': job_func,
                         'Industry': industries
                         })
job_data.to_csv('linkedin_jobs_data-3.csv', index=False)

In [248]:
# Load further details
from time import sleep


filename = 'linkedin_jobs_data-3.csv'
# Opening csv file to read job links

with open(filename, 'r') as csvfile: 
    csvreader = csv.reader(csvfile)
    next(csvreader)

    # Indicates which job is going to be updated
    CSVtabelIndex = 0

    for job in csvreader:
        
        url = job[5]
        print("Current Scraped Url:\n",url)
        wd.get(url)

        print("DEBUG: ", wd.current_url)

        # Loop verifying if a redirect has occurred and then undoes it.
        while 'authwall' in wd.current_url:
            print(wd.current_url)
            time.sleep(2)
            wd.get(url)

        # Selecting elements that hold the job description (about)
        jd0 = wd.find_element(By.CLASS_NAME, value='show-more-less-html__markup').get_attribute("innerHTML")
        print(jd0)

        # Selecting elements that hold job 4 criterias 
        job = wd.find_elements(By.CLASS_NAME, value='description__job-criteria-item')

        seniority0 = job[0].find_element(By.CLASS_NAME, value='description__job-criteria-text').text
        
        emp_type0 = job[1].find_element(By.CLASS_NAME, value='description__job-criteria-text').text
        
        job_func0 = job[2].find_element(By.CLASS_NAME, value='description__job-criteria-text').text

        industries0 = job[3].find_element(By.CLASS_NAME, value='description__job-criteria-text').text

        # Adding new information to the existing jobs 
        jd[CSVtabelIndex] = jd0
        seniority[CSVtabelIndex] = seniority0
        emp_type[CSVtabelIndex]= emp_type0
        job_func[CSVtabelIndex] = job_func0         
        industries[CSVtabelIndex] = industries0

        CSVtabelIndex = CSVtabelIndex + 1
        
        time.sleep(10)


Current Scraped Url:
 https://br.linkedin.com/jobs/view/analista-de-processos-foco-em-analise-de-dados-at-incentiv-me-3272108081?refId=M3gvRWmiPk9RfbiFbr9yRA%3D%3D&trackingId=fDC%2BEqFkaEXoPx9JoEAyMg%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card
DEBUG:  https://br.linkedin.com/jobs/view/analista-de-processos-foco-em-analise-de-dados-at-incentiv-me-3272108081?refId=M3gvRWmiPk9RfbiFbr9yRA%3D%3D&trackingId=fDC%2BEqFkaEXoPx9JoEAyMg%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card

        Como Analista de Processos Pleno você terá a missão de contribuir para implementação e manutenção do sistema de gestão de processos de negócios e qualidade transformando as fontes de dados em informações, facilitando a tomada de decisão dos gestores da empresa, garantindo a aderência aos parâmetros das normas e exigências do mercado, dentro das políticas definidas pela empresa.<br><br><strong><u>Então Se Liga<br><br></u></strong>Nesse sentido precisamos que você te

KeyboardInterrupt: 

In [ ]:
#Load data into DataFrame for further analysis
job_data = pd.DataFrame({'ID': job_id,
                         'Date': date,
                         'Company': company_name,
                         'Title': job_title,
                         'Location': location,
                         'Link': job_link,
                         'Description': jd,
                         'Level': seniority,
                         'Type': emp_type,
                         'Function': job_func,
                         'Industry': industries
                         })

job_data.to_csv('linkedin_jobs_data-3.csv', index=False)